In [1]:
import pandas as pd
import re
import numpy as np
import random

# Load parameters from file
parameters = [
    'toxe', 'toxm', 'toxref','toxp', 'xj', 'ndep'
]

### 1.1.3 Function to generate random values within specified ranges

In [2]:
def generate_random_values():
    # Supply voltage with ±10% variations
    supply = random.uniform(0.9,1.1)
    return supply

In [4]:
# Define the number of samples
num_samples = 10000
lmin = 45
wmin = 45

# Read the PTM file
with open("45nm_MGK.pm", 'r') as file:
    original_content = file.read()
    
temp_cqload_df=pd.read_csv('../Temp_Cqload.csv')

# Extract nominal values for parameters
nominal_values = {}
for param in parameters:
    original_values = re.findall(rf'\b{param}\s*=\s*([\d.e+-]+)', original_content)
    nominal_values[param] = [float(value) for value in original_values]


# Calculate standard deviation (std) for each parameter
std_values = {key: [value / 30 for value in values] for key, values in nominal_values.items()}

# Generate random samples with Monte Carlo distribution for parameters for NMOS and PMOS
nmos_parameters=pmos_parameters=parameters

nmos_samples_data = {param: np.random.normal(loc=nominal_values[param][0], scale=std_values[param][0], size=num_samples) for param in nmos_parameters}
pmos_samples_data = {param: np.random.normal(loc=nominal_values[param][1], scale=std_values[param][1], size=num_samples) for param in pmos_parameters}

# Calculate mean and std for lmin and wmin
lmin_mean = 47
lmin_std = lmin_mean / 30
wmin_mean = 254
wmin_std = wmin_mean / 30

# Generate random samples for Lmin and Wmin with Monte Carlo distribution
lmin_samples = np.random.normal(loc=lmin_mean, scale=lmin_std, size=num_samples)
wmin_samples = np.random.normal(loc=wmin_mean, scale=wmin_std, size=num_samples)

# Generate samples for temp, supply, and cqload
temp_samples = []
supply_samples = []
cqload_samples = []
for i in range(num_samples):
    supply = generate_random_values()
    temp_samples.append(temp_cqload_df['temp'][i])
    supply_samples.append(supply)
    cqload_samples.append(temp_cqload_df['cqload'][i])
    
temp_nmos_samples_data,temp_pmos_samples_data=nmos_samples_data,pmos_samples_data

# Add '_n' suffix to keys in nmos_samples_data
temp_nmos_samples_data = {key + '_n': value for key, value in temp_nmos_samples_data.items()}

# Add '_p' suffix to keys in pmos_samples_data
temp_pmos_samples_data = {key + '_p': value for key, value in temp_pmos_samples_data.items()}

# Create a DataFrame
df_samples = pd.DataFrame({
    'lmin': lmin_samples,
    'wmin': wmin_samples,
    'temp': temp_samples,
    'supply': supply_samples,
    'cqload': cqload_samples,
    **temp_nmos_samples_data,
    **temp_pmos_samples_data
})


In [6]:
df_samples.to_csv('samples_45nm_MGK.csv')